<a href="https://colab.research.google.com/github/ameyanjarlekar/Ball-Detection-Updated/blob/master/Sparse_Attentive_Inapinting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn

class AlexEnc_Dec(nn.Module):
    def __init__(self, num_classes=1000):
        super(AlexEnc_Dec, self).__init__()
        #self.Encoder = nn.Sequential(
        #    nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
        #    nn.ReLU(inplace=True),
        #    nn.MaxPool2d(kernel_size=3, stride=2,return_indices=True),
        #    nn.Conv2d(64, 192, kernel_size=5, padding=2),
        #    nn.ReLU(inplace=True),
        #    nn.MaxPool2d(kernel_size=3, stride=2,return_indices=True),
        #    nn.Conv2d(192, 384, kernel_size=3, padding=1),
        #    nn.ReLU(inplace=True),
        #    nn.Conv2d(384, 256, kernel_size=3, padding=1),
        #    nn.ReLU(inplace=True),
        #    nn.Conv2d(256, 256, kernel_size=3, padding=1),
        #    nn.ReLU(inplace=True),
        #    nn.MaxPool2d(kernel_size=3, stride=2,return_indices=True),
        #)
        self.Decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxUnpool2d(kernel_size=3, stride=2),
            nn.ConvTranspose2d(256, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(384, 192, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(192, 64, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxUnpool2d(kernel_size=3, stride=2),
            nn.ConvTranspose2d(64, 3, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxUnpool2d(kernel_size=3, stride=2),
        )

    def forward(self, x):
        x = self.Encoder(x)
        x = self.Decoder(x)
        return x
AlexEnc_Dec = AlexEnc_Dec()

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import torch, os
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as dset
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.utils import save_image
from torchvision.utils import make_grid
from torch.autograd import Variable
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
%matplotlib inline

In [0]:
lr = 0.0002
max_epoch = 8
batch_size = 32
z_dim = 100
image_size = 128
g_conv_dim = 68
d_conv_dim = 63
log_step = 100
sample_step = 500
sample_num = 32


In [0]:
transform = transforms.Compose([
    #transforms.Scale(64),
    # transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

dataset = ImageFolder("//content/datareqd", transform)
data_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=16, drop_last=True)

In [0]:
!unzip "/content/drive/My Drive/img_align_celeba.zip"

In [0]:
ls "/content/img_align_celeba"

In [11]:
import torch.optim as optim
criterion = torch.nn.SmoothL1Loss
optimizer = optim.SGD(AlexEnc_Dec.parameters(), lr=0.01, momentum=0.9)
print("hi")
for epoch in range(4):  # loop over the dataset multiple times
    print("in")
    running_loss = 0.0
    for i, (data,val) in enumerate(data_loader):
        print("oy")
        print(data.size())
        print(type(data))
        # get the inputs; data is a list of [inputs, labels]
        #inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()
        data = torch.tensor(data)

        # forward + backward + optimize
        outputs = AlexEnc_Dec(data)
        loss = criterion(outputs, labels)
        print(loss)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

hi
in
oy
torch.Size([32, 3, 218, 178])
<class 'torch.Tensor'>


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


TypeError: ignored

In [0]:
!mkdir datareqd